<a href="https://colab.research.google.com/github/chaerui7967/stock_predict_news_and_youtube/blob/master/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90_%ED%86%B5%ED%95%A9%ED%85%8C%EC%9D%B4%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리 설치

In [1]:
!pip install finance-datareader
!pip install pykrx

     |████████████████████████████████| 81 kB 3.8 MB/s 
     |████████████████████████████████| 60 kB 5.9 MB/s 
     |████████████████████████████████| 251 kB 16.8 MB/s 


## 패키지 & 함수 로딩

In [2]:
# 필요 라이브러리
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 주식가격 로딩
import FinanceDataReader as fdr
from pykrx import stock

import datetime

In [3]:
def money_sur(stock_name, start, end):

    start_date = start
    end_date = end
    sample_code = stock_name

    dataset = fdr.DataReader(sample_code, start = start_date, end = end_date )
    dataset = dataset.reset_index()
    df = stock.get_market_fundamental_by_date(fromdate=start_date, todate=end_date, ticker=sample_code)
    df = df.reset_index()
    df.columns = ['Date','BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']
    data = pd.merge(dataset, df, on='Date')

    return data

In [4]:
def reverse_min_max_scaling(org_x, x): #종가 예측값
    org_x_np = np.asarray(org_x) 
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min())) + org_x_np.min()

## 통합테이블 생성

In [19]:
dic = {'code' : [],
       'date' : [],
       'lstm' : [],
       'arima' : [],
       'fbprophet' : [],
       'news_text' : [],
       'nsi' : [],
       'youtube_text' : [],
       'close' : []}

total_table = pd.DataFrame(dic)

In [20]:
total_table

,code,date,lstm,arima,fbprophet,news_text,nsi,youtube_text,close


## LSTM 예측결과

### 사용 데이터 로드 및 전처리

In [66]:
start_date = (datetime.datetime.now() - datetime.timedelta(30)).strftime("%Y%m%d")
end_date = datetime.datetime.now().strftime("%Y%m%d")
lstm = money_sur('005930', start_date, end_date)

In [67]:
lstm = lstm.set_index('Date')

In [68]:
lstm.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS
Date,,,,,,,,,,,,
2021-08-31,74900,76700,74300,76700,24630370,0.028150,39406,19.97,1.95,3841,3.90,2994
2021-09-01,76700,77100,75900,76800,16114775,0.001304,39406,19.99,1.95,3841,3.90,2994
2021-09-02,76800,76800,75700,76000,15347486,-0.010417,39406,19.79,1.93,3841,3.94,2994
2021-09-03,76400,76700,76000,76600,12096419,0.007895,39406,19.94,1.94,3841,3.91,2994
2021-09-06,76800,77600,76600,77300,12861180,0.009138,39406,20.12,1.96,3841,3.87,2994


In [69]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
lstm_sc = sc.fit_transform(lstm)

In [70]:
scale_cols = lstm.columns
lstm_sc_df = pd.DataFrame(lstm_sc, columns=[scale_cols], index=lstm.index)

In [71]:
lstm_sc_df.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS
Date,,,,,,,,,,,,
2021-08-31,0.176471,0.633333,0.176471,0.722222,1.000000,1.000000,0.0,0.723404,0.777778,0.0,0.263159,0.0
2021-09-01,0.705882,0.766667,0.647059,0.750000,0.413815,0.528877,0.0,0.744681,0.777778,0.0,0.263159,0.0
2021-09-02,0.735294,0.666667,0.588235,0.527778,0.360998,0.323196,0.0,0.531915,0.555555,0.0,0.473685,0.0
2021-09-03,0.617647,0.633333,0.676471,0.694444,0.137206,0.644540,0.0,0.691489,0.666667,0.0,0.315790,0.0
2021-09-06,0.735294,0.933333,0.852941,0.888889,0.189849,0.666365,0.0,0.882979,0.888889,0.0,0.105263,0.0


In [72]:
for i in scale_cols:
  for s in range(1, 11):  
    lstm_sc_df[(i+'shift_{}'.format(s),)] = lstm_sc_df[(i,)].shift(s)

In [73]:
lstm_sc_df.dropna()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS,Openshift_1,Openshift_2,Openshift_3,Openshift_4,Openshift_5,Openshift_6,Openshift_7,Openshift_8,Openshift_9,Openshift_10,Highshift_1,Highshift_2,Highshift_3,Highshift_4,Highshift_5,Highshift_6,Highshift_7,Highshift_8,Highshift_9,Highshift_10,Lowshift_1,Lowshift_2,Lowshift_3,Lowshift_4,Lowshift_5,Lowshift_6,Lowshift_7,Lowshift_8,...,PBRshift_1,PBRshift_2,PBRshift_3,PBRshift_4,PBRshift_5,PBRshift_6,PBRshift_7,PBRshift_8,PBRshift_9,PBRshift_10,EPSshift_1,EPSshift_2,EPSshift_3,EPSshift_4,EPSshift_5,EPSshift_6,EPSshift_7,EPSshift_8,EPSshift_9,EPSshift_10,DIVshift_1,DIVshift_2,DIVshift_3,DIVshift_4,DIVshift_5,DIVshift_6,DIVshift_7,DIVshift_8,DIVshift_9,DIVshift_10,DPSshift_1,DPSshift_2,DPSshift_3,DPSshift_4,DPSshift_5,DPSshift_6,DPSshift_7,DPSshift_8,DPSshift_9,DPSshift_10
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-09-14,0.823529,0.966667,0.852941,0.694444,0.555088,0.574996,0.0,0.691489,0.666667,0.0,0.315790,0.0,0.264706,0.294118,0.617647,0.500000,0.823529,0.735294,0.617647,0.735294,0.705882,0.176471,0.500000,0.266667,0.600000,0.533333,0.766667,0.933333,0.633333,0.666667,0.766667,0.633333,0.411765,0.323529,0.382353,0.558824,0.647059,0.852941,0.676471,0.588235,...,0.666667,0.333333,0.333333,0.666667,0.555555,0.888889,0.666667,0.555555,0.777778,0.777778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.368422,0.684211,0.684211,0.368422,0.421053,0.105263,0.315790,0.473685,0.263159,0.263159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-15,0.911765,0.866667,0.794118,0.805556,0.187643,0.597636,0.0,0.808511,0.777778,0.0,0.210527,0.0,0.823529,0.264706,0.294118,0.617647,0.500000,0.823529,0.735294,0.617647,0.735294,0.705882,0.966667,0.500000,0.266667,0.600000,0.533333,0.766667,0.933333,0.633333,0.666667,0.766667,0.852941,0.411765,0.323529,0.382353,0.558824,0.647059,0.852941,0.676471,...,0.666667,0.666667,0.333333,0.333333,0.666667,0.555555,0.888889,0.666667,0.555555,0.777778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.315790,0.368422,0.684211,0.684211,0.368422,0.421053,0.105263,0.315790,0.473685,0.263159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-16,0.882353,0.866667,0.705882,0.555556,0.204060,0.300880,0.0,0.553191,0.555555,0.0,0.421053,0.0,0.911765,0.823529,0.264706,0.294118,0.617647,0.500000,0.823529,0.735294,0.617647,0.735294,0.866667,0.966667,0.500000,0.266667,0.600000,0.533333,0.766667,0.933333,0.633333,0.666667,0.794118,0.852941,0.411765,0.323529,0.382353,0.558824,0.647059,0.852941,...,0.777778,0.666667,0.666667,0.333333,0.333333,0.666667,0.555555,0.888889,0.666667,0.555555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.210527,0.315790,0.368422,0.684211,0.684211,0.368422,0.421053,0.105263,0.315790,0.473685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-17,0.588235,0.800000,0.647059,0.861111,0.425843,0.759660,0.0,0.861702,0.888889,0.0,0.157896,0.0,0.882353,0.911765,0.823529,0.264706,0.294118,0.617647,0.500000,0.823529,0.735294,0.617647,0.866667,0.866667,0.966667,0.500000,0.266667,0.600000,0.533333,0.766667,0.933333,0.633333,0.705882,0.794118,0.852941,0.411765,0.323529,0.382353,0.558824,0.647059,...,0.555555,0.777778,0.666667,0.666667,0.333333,0.333333,0.666667,0.555555,0.888889,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.421053,0.210527,0.315790,0.368422,0.684211,0.684211,0.368422,0.421053,0.105263,0.315790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-23,0.941176,0.933333,0.911765,0.916667,0.478578,0.551460,0.0,0.914894,0.888889,0.0,0.105263,0.0,0.588235,0.882353,0.911765,0.823529,0.264706,0.294118,0.617647,0.500000,0.823529,0.735294,0.800000,0.866667,0.866667,0.966667,0.500000,0.266667,0.600000,0.533333,0.766667,0.933333,0.647059,0.705882,0.794118,0.852941,0.411765,0.323529,0.382353,0.558824,...,0.888889,0.555555,0.777778,0.666667,0.666667,0.333333,0.333333,0.666667,0.555555,0.888889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157896,0.421053,0.210527,0.315790,0.368422,0.684211,0.684211,0.368422,0.421053,0.105263,0.0,0.0,0.0,0.0,0

In [74]:
X_test = lstm_sc_df.dropna().drop(columns=scale_cols, axis=1)
y_test = lstm_sc_df.dropna()[['Close']]

X_test= X_test.values
y_test = y_test.values

In [ ]:
X_test

In [76]:
X_test_t = X_test.reshape(X_test.shape[0], 10, 12)

### 모델 로드

In [77]:
# 데이터 불러오기
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()

model.add(LSTM(64,
               input_shape = (10, 12),
               return_sequences= True))
model.add(Dense(32, activation = 'relu'))
model.add(LSTM(64,
               input_shape = (10, 12),
               return_sequences= True))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'tanh'))
model.add(LSTM(64,
               return_sequences = False))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1))



lstm_path = '/content/drive/My Drive/Final PJT - 업빛투/Regression/LSTM/'
samsung_lstm = 'lstm_samsung.h5'

model.load_weights(lstm_path+samsung_lstm)

### 모델 결과

In [79]:
pred = model.predict(X_test_t)

In [81]:
pred.shape, y_test.shape

((10, 1), (10, 1))

In [83]:
pred_act = reverse_min_max_scaling(lstm['Close'], pred)

In [98]:
total_table['date'] = lstm.index[-10:]
total_table['lstm'] = pred_act
total_table['close'] = lstm.Close[-10:].values.reshape(-1,1)
total_table['code'] = '005930'

In [100]:
total_table

,code,date,lstm,arima,fbprophet,news_text,nsi,youtube_text,close
0,005930,2021-09-14,75607.274414,NaN,NaN,NaN,NaN,NaN,76600
1,005930,2021-09-15,75569.954956,NaN,NaN,NaN,NaN,NaN,77000
2,005930,2021-09-16,75628.717407,NaN,NaN,NaN,NaN,NaN,76100
3,005930,2021-09-17,76019.176758,NaN,NaN,NaN,NaN,NaN,77200
4,005930,2021-09-23,76279.769043,NaN,NaN,NaN,NaN,NaN,77400
5,005930,2021-09-24,76373.806885,NaN,NaN,NaN,NaN,NaN,77300
6,005930,2021-09-27,76342.911621,NaN,NaN,NaN,NaN,NaN,77700
7,005930,2021-09-28,76418.090332,NaN,NaN,NaN,NaN,NaN,76300
8,005930,2021-09-29,76758.200684,NaN,NaN,NaN,NaN,NaN,74100
9,005930,2021-09-30,76251.181396,NaN,NaN,NaN,NaN,NaN,74600


## arima 예측결과

In [162]:
end_date = datetime.datetime.now().strftime("%Y%m%d")
samsung_df = money_sur('005930', '2018', end_date)

In [163]:
df = samsung_df[['Date','Close']]
df.columns = ['Date', 'Price']

In [166]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
# 학습, 테스트 데이터 분리
train = df[:-1]
test = df[-1:]

model = ARIMA(train.Price.values, order = (2,1,2))
model_fit = model.fit(trend = 'nc', full_output = True, disp = True)
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                  921
Model:                 ARIMA(2, 1, 2)   Log Likelihood               -7614.762
Method:                       css-mle   S.D. of innovations            940.506
Date:                Thu, 30 Sep 2021   AIC                          15239.524
Time:                        07:32:03   BIC                          15263.651
Sample:                             1   HQIC                         15248.731
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1.D.y     -1.5978      0.010   -156.898      0.000      -1.618      -1.578
ar.L2.D.y     -0.9812      0.008   -121.792      0.000      -0.997      -0.965
ma.L1.D.y      1.5992      0.009    187.126      0.0

In [167]:
fore = model_fit.forecast(steps=1) # 오늘 예측

# 오늘 예측 데이터
pred_arima_y = fore[0].tolist()

# 실제 오늘 데이터 
test_y = test.Price.values

# # 신뢰구간
# # 예측 데이터 최소값
# pred_y_lower = []
# # 예측 데이터 최대값
# pred_y_upper = []

# for lower_upper in fore[2]:
#     lower = lower_upper[0]
#     upper = lower_upper[1]
#     pred_y_lower.append(lower)
#     pred_y_upper.append(upper)

In [180]:
total_table['arima'].iloc[-1:] = pred_arima_y

In [ ]:
# 10일치를 넣기 위한
# for i in [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10]:
#   train = df[:i]
#   test = df[i:i+1]

#   model = ARIMA(train.Price.values, order = (2,1,2))
#   model_fit = model.fit(trend = 'nc', full_output = True, disp = True)

#   fore = model_fit.forecast(steps=1)
#   pred_arima_y = fore[0].tolist()

#   total_table['arima'].iloc[i:i+1] = pred_arima_y

In [193]:
total_table

,code,date,lstm,arima,fbprophet,news_text,nsi,youtube_text,close
0,005930,2021-09-14,75607.274414,76546.648330,76795.828122,NaN,NaN,NaN,76600
1,005930,2021-09-15,75569.954956,76491.084975,76752.859279,NaN,NaN,NaN,77000
2,005930,2021-09-16,75628.717407,76936.879734,76786.923009,NaN,NaN,NaN,76100
3,005930,2021-09-17,76019.176758,76320.417183,76795.446254,NaN,NaN,NaN,77200
4,005930,2021-09-23,76279.769043,76895.109581,76709.397125,NaN,NaN,NaN,77400
5,005930,2021-09-24,76373.806885,77680.410699,76605.655722,NaN,NaN,NaN,77300
6,005930,2021-09-27,76342.911621,77162.591663,76326.442207,NaN,NaN,NaN,77700
7,005930,2021-09-28,76418.090332,77641.180383,76391.288487,NaN,NaN,NaN,76300
8,005930,2021-09-29,76758.200684,76541.424791,76163.200255,NaN,NaN,NaN,74100
9,005930,2021-09-30,76251.181396,73766.224648,76026.706562,NaN,NaN,NaN,74600


## fbprophet 예측

In [215]:
end_date = datetime.datetime.now().strftime("%Y%m%d")
samsung_df = money_sur('005930', start = '20180101', end = end_date)
df1 = samsung_df[['Date','Close']]
df1.columns = ['ds', 'y']

from fbprophet import Prophet

prophet = Prophet(seasonality_mode = 'multiplicative',
                 yearly_seasonality=True, 
                 weekly_seasonality=True,
                 daily_seasonality=True,
                 changepoint_prior_scale=0.6)

prophet.fit(df1)

# 10일단위로 예측값을 가져옴
future_data = prophet.make_future_dataframe(periods = 10, freq = 'd')
forecast_data = prophet.predict(future_data)

In [216]:
list(total_table.date.values)

[numpy.datetime64('2021-09-14T00:00:00.000000000'),
 numpy.datetime64('2021-09-15T00:00:00.000000000'),
 numpy.datetime64('2021-09-16T00:00:00.000000000'),
 numpy.datetime64('2021-09-17T00:00:00.000000000'),
 numpy.datetime64('2021-09-23T00:00:00.000000000'),
 numpy.datetime64('2021-09-24T00:00:00.000000000'),
 numpy.datetime64('2021-09-27T00:00:00.000000000'),
 numpy.datetime64('2021-09-28T00:00:00.000000000'),
 numpy.datetime64('2021-09-29T00:00:00.000000000'),
 numpy.datetime64('2021-09-30T00:00:00.000000000')]

In [217]:
total_table['fbprophet'] = forecast_data[forecast_data['ds'].isin(list(total_table.date.values))]['yhat'].values

In [218]:
total_table

,code,date,lstm,arima,fbprophet,news_text,nsi,youtube_text,close
0,005930,2021-09-14,75607.274414,76546.648330,76795.828122,NaN,NaN,NaN,76600
1,005930,2021-09-15,75569.954956,76491.084975,76752.859279,NaN,NaN,NaN,77000
2,005930,2021-09-16,75628.717407,76936.879734,76786.923009,NaN,NaN,NaN,76100
3,005930,2021-09-17,76019.176758,76320.417183,76795.446254,NaN,NaN,NaN,77200
4,005930,2021-09-23,76279.769043,76895.109581,76709.397125,NaN,NaN,NaN,77400
5,005930,2021-09-24,76373.806885,77680.410699,76605.655722,NaN,NaN,NaN,77300
6,005930,2021-09-27,76342.911621,77162.591663,76326.442207,NaN,NaN,NaN,77700
7,005930,2021-09-28,76418.090332,77641.180383,76391.288487,NaN,NaN,NaN,76300
8,005930,2021-09-29,76758.200684,76541.424791,76163.200255,NaN,NaN,NaN,74100
9,005930,2021-09-30,76251.181396,73766.224648,76026.706562,NaN,NaN,NaN,74600


## News 긍부정 결과

In [219]:
total_table.to_csv('./total_table.csv', index =False)